In [ ]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime
import math
from datetime import timedelta
import random
import warnings
import os
import pandas as pd
import time
from multiprocessing.pool import ThreadPool as Pool
warnings.filterwarnings("ignore")
pool_size = 10 # your "parallelness"
pool = Pool(pool_size)


In [ ]:
1+9

In [2]:
file_directory='tick_data/'
file_directory1='VwapPrediction'
list_of_files=os.listdir(file_directory)

counter_file=0
path_for_results='newResults/'


k='ESc1.csv'

In [3]:
params = {
    "stoploss_tick_inside": 5,
    "time_interval": 15,
    "time_interval_text": '15T',
    "pnl_stop": -250,
    "secondary_pnl_stop": -200,
    "max_stoploss": 4,
    "l": 4,
    "stoploss_tick":37,
    "entry_level":5,
    "top_level":8
}
params_list=list(params.values())

In [7]:

for k in list_of_files[0:1]:
    
    start=time.time()
    filename = file_directory + k
    df_raw = pd.read_csv(
        filename, usecols=['Date-Time', 'GMT Offset', 'Price', 'Volume', 'Bid Price', 'Ask Price'],
        dtype={'Price': np.float32, 'Volume': 'Int32'}
    )
    
    df_raw.dropna(inplace=True)
    df_raw['GMT Offset'] += 5
    df_raw['Date-Time'] = pd.to_datetime(df_raw['Date-Time']) + pd.to_timedelta(df_raw['GMT Offset'], unit='h')
    df_raw.set_index('Date-Time', inplace=True)
    df_raw['price_vol'] = df_raw['Price'] * df_raw['Volume']

    # Load predicted VWAP data
    data_vwap = pd.read_excel('predictNQZ22_15M.xlsx')
    data_vwap.dropna(inplace=True)

    # Align datetime formats
    df_raw.reset_index(inplace=True)
    df_raw['Time'] = df_raw['Date-Time'].values.astype('datetime64[s]')
    df_raw = df_raw[df_raw[['Bid Price', 'Ask Price']].diff().any(axis=1)]
    
    data_vwap['Time'] = pd.to_datetime(data_vwap['Time'])


    data_vwap=data_vwap.dropna()

    pool = Pool(pool_size)
    tick_size_dir={'CL':[.01,30],
    'FDX':[1,10],
    'WDO':[.5,30],
    'WIN':[5,10],
    'ES':[.25,50]
                }
    list_files={'CLF3-H3.csv':[.01,1,15],
    'CLH3-M3.csv':[.01,1,10],
    'CLZ2-F3.csv':[.01,1,5],
    'CLZ2-M3.csv':[.01,1,10],
    'CLZ2-Z3.csv':[.01,2,5],
    'LCOF3-H3.csv':[.01,1,5],
    'LCOH3-M3.csv':[.01,1,10],
    'LCOH3-N3.csv':[.01,1,10],
    'LCOJ3-K3.csv':[.01,1,10],
    'LCOM3-Z3.csv':[.01,2,5],
    'LCOZ2-F3.csv':[.01,1,5],
    'LCOZ2-M3.csv':[.01,1,10],
    'LCOZ2-Z3.csv':[.01,2,15],
    'CLc1-CLc2.csv':[.01,2,15],
    'ESc1.csv':[.25,params_list[8],params_list[9]]}
    m=k
    tick_size=list_files[m][0]
    max_level=list_files[m][2]
    level_tick=list_files[m][1]
    df=df_raw

    data=data_vwap

    data['Only_Date']=data['Time'].values.astype('datetime64[D]')

    Only_Date=data['Only_Date'].unique()

    column_names = ['hour',"Type", "Entry Index", 'Entry Time', "Entry Price",'level',"Exit Index","Exit Time","Exit Price","Predicted","Trade_close"]

    buy1=0
    sell1=0
    stoploss_tick=params_list[7] # it should be variable

    #temp_tradelog = pd.DataFrame(columns = column_names,index=range(500000))
    temp_trade_count=0
    global_tradeLog=pd.DataFrame(columns=column_names)
    
    
    def collect_result(check):
        global global_tradeLog
        global_tradeLog=pd.concat([global_tradeLog, check],ignore_index=True)

    print("\n*********************************************\n")
    for date_thr in Only_Date:
        #day_trade=trade_logic(day_data,m)
        temp_date=pd.to_datetime(date_thr)

        day_data=(data[(data['Only_Date']==temp_date)])
        pool.apply_async(trade_logic, (day_data,m,stoploss_tick,list_files,),callback=collect_result)


    print("\n*********************************************\n")
    pool.close()
    pool.join() 





    global_tradeLog=global_tradeLog.sort_values(by='Entry Time',ascending=True)
    global_tradeLog.loc[global_tradeLog['Type']=='buy',"Pnl"]=((global_tradeLog[global_tradeLog['Type']=='buy']['Exit Price']-global_tradeLog[global_tradeLog['Type']=='buy']['Entry Price'])/tick_size)
    global_tradeLog.loc[global_tradeLog['Type']=='sell',"Pnl"]=((global_tradeLog[global_tradeLog['Type']=='sell']['Entry Price']-global_tradeLog[global_tradeLog['Type']=='sell']['Exit Price'])/tick_size)
    global_tradeLog.loc[global_tradeLog['Pnl']>0,"Win"]=1
    global_tradeLog.loc[global_tradeLog['Pnl']<=0,"Win"]=0  
    global_tradeLog=global_tradeLog.sort_values(by='Entry Time',ascending=True)
    
    #global_tradeLog.to_csv('yearly_8_5_es.csv')

    global_tradeLog['drawdown']=''
    timer4=time.time()
    for i in global_tradeLog.itertuples(name=None):
        entry_time=i[4]
        index=i[0]
        exit_time=i[8]
        if i[2]=='sell':
            temp_data=(df[((df['Date-Time']>=entry_time) & (df['Date-Time']<=exit_time))])
            global_tradeLog['drawdown'][index]=(round(temp_data['Price'].max()/tick_size))*tick_size
        else:

            temp_data=(df[((df['Date-Time']>=entry_time) & (df['Date-Time']<=exit_time))])
            global_tradeLog['drawdown'][index]=(round(temp_data['Price'].min()/tick_size))*tick_size


    global_tradeLog.loc[global_tradeLog['Type']=='sell',"drawdown_tick"]=((global_tradeLog[global_tradeLog['Type']=='sell']['drawdown']-global_tradeLog[global_tradeLog['Type']=='sell']['Entry Price'])/tick_size)
    global_tradeLog.loc[global_tradeLog['Type']=='buy',"drawdown_tick"]=((global_tradeLog[global_tradeLog['Type']=='buy']['Entry Price']-global_tradeLog[global_tradeLog['Type']=='buy']['drawdown'])/tick_size)
    tradelog=global_tradeLog.dropna()
    #tradelog.to_csv('I:\GGNQuants\VWAP_STDEV/results/tradelog_vwap'+str(comb[0])+'_sd_'+str(comb[1])+str(m))
    #tradelog=tradelog[(tradelog['Trade_close']!='Predicted')]




#************************************************************** Backtesting Started ***********************************************************




    index_list=["No. of Positive Trades All",
    "No. of Negative Trades All",
    "Win percent All",
    "Total Trades All",
    "Maximum Value All",
    "Minimum Value All",
    "Win Streak",
    "Losing Streak",
    "Average Drawdown",
    "Average Win Drawdown",
    "Total Profit All",
    "Average Profit All",
    "No. of Positive Trades All_asia",
    "No. of Negative Trades All_asia",
    "Win percent All_asia",
    "Total Trades All_asia",
    "Maximum Value All_asia",
    "Minimum Value All_asia",
    "Win Streak_asia",
    "Losing Streak_asia",
    "Average Drawdown_asia",
    "Average Win Drawdown_asia",
    "Total Profit All_asia",
    "Average Profit All_asia",
    "No. of Positive Trades All_uk",
    "No. of Negative Trades All_uk",
    "Win percent All_uk",
    "Total Trades All_uk",
    "Maximum Value All_uk",
    "Minimum Value All_uk",
    "Win Streak_uk",
    "Losing Streak_uk",
    "Average Drawdown_uk",
    "Average Win Drawdown_uk",
    "Total Profit All_uk",
    "Average Profit All_uk",
    "No. of Positive Trades All_us",
    "No. of Negative Trades All_us",
    "Win percent All_us",
    "Total Trades All_us",
    "Maximum Value All_us",
    "Minimum Value All_us",
    "Win Streak_us",
    "Losing Streak_us",
    "Average Drawdown_us",
    "Average Win Drawdown_us",
    "Total Profit All_us",
    "Average Profit All_us"
            ]


    level_list=sorted((tradelog['level'].unique()).tolist())
    backtest_lstm = pd.DataFrame(index=index_list,columns=level_list)
    tradelog_asia=tradelog[(tradelog['hour']<=8) & (tradelog['hour']>=0)]
    tradelog_uk=tradelog[(tradelog['hour']>8) & (tradelog['hour']<=14)]
    tradelog_us=tradelog[(tradelog['hour']>14) & (tradelog['hour']<=23)]



    for i in level_list:
        temp_back=tradelog[tradelog['level']==i]

        Win_arr = np.array(temp_back['Win'])
        number_of_positive_trades = np.sum(Win_arr)


        #Finding negative trades
        number_of_negative_trades=len(temp_back['Win'])-number_of_positive_trades

        #Finding total trades
        total_trades=len(temp_back['Win'])


        #Finding max profit in all trades
        max_value = temp_back['Pnl'].max()
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value)

        #Finding min profit in all trades
        min_value = temp_back['Pnl'].min()


        temp_back['win']=np.where(temp_back['Pnl']>=0, 1, 0)
        #df3=df3.dropna()
    #     print(len(df3))
        win=temp_back['win']
        win_streak = (win.groupby((win != win.shift(1)).cumsum()).cumsum().max())
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value,min_value,win_streak)
        average_drawdown=temp_back['drawdown_tick'].mean()
        average_drawdown_win=temp_back[temp_back['Win']==1]['drawdown_tick'].mean()
        
        temp_back['loss']=np.where(temp_back['Pnl']<0, 1, 0)
        loss=temp_back['loss']
        losing_streak = (loss.groupby((loss != loss.shift(1)).cumsum()).cumsum().max())
        mean=temp_back['Pnl'].mean()
        sum_t=temp_back['Pnl'].sum()

        ###############################
        temp_back_asia=tradelog_asia[tradelog_asia['level']==i]

        Win_arr_asia = np.array(temp_back_asia['Win'])
        number_of_positive_trades_asia = np.sum(Win_arr_asia)


        #Finding negative trades
        number_of_negative_trades_asia=len(temp_back_asia['Win'])-number_of_positive_trades_asia

        #Finding total trades
        total_trades_asia=len(temp_back_asia['Win'])


        #Finding max profit in all trades
        max_value_asia = temp_back_asia['Pnl'].max()
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value)

        #Finding min profit in all trades
        min_value_asia = temp_back_asia['Pnl'].min()


        temp_back_asia['win']=np.where(temp_back_asia['Pnl']>=0, 1, 0)
        #df3=df3.dropna()
    #     print(len(df3))
        win_asia=temp_back_asia['win']
        win_streak_asia = (win.groupby((win_asia != win_asia.shift(1)).cumsum()).cumsum().max())
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value,min_value,win_streak)
        average_drawdown_asia=temp_back_asia['drawdown_tick'].mean()
        average_drawdown_win_asia=temp_back_asia[temp_back_asia['Win']==1]['drawdown_tick'].mean()
        temp_back_asia['loss']=np.where(temp_back_asia['Pnl']<0, 1, 0)
        loss_asia=temp_back_asia['loss']
        losing_streak_asia = (loss.groupby((loss_asia != loss_asia.shift(1)).cumsum()).cumsum().max())
        mean_asia=temp_back_asia['Pnl'].mean()
        sum_t_asia=temp_back_asia['Pnl'].sum()

        #####################################
        temp_back_uk=tradelog_uk[tradelog_uk['level']==i]

        Win_arr_uk = np.array(temp_back_uk['Win'])
        number_of_positive_trades_uk = np.sum(Win_arr_uk)


        #Finding negative trades
        number_of_negative_trades_uk=len(temp_back_uk['Win'])-number_of_positive_trades_uk

        #Finding total trades
        total_trades_uk=len(temp_back_uk['Win'])


        #Finding max profit in all trades
        max_value_uk = temp_back_uk['Pnl'].max()
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value)

        #Finding min profit in all trades
        min_value_uk = temp_back_uk['Pnl'].min()


        temp_back_uk['win']=np.where(temp_back_uk['Pnl']>=0, 1, 0)
        #df3=df3.dropna()
    #     print(len(df3))
        win_uk=temp_back_uk['win']
        win_streak_uk = int(win.groupby((win_uk != win_uk.shift(1)).cumsum()).cumsum().max())
        average_drawdown_uk=temp_back_uk['drawdown_tick'].mean()
        average_drawdown_win_uk=temp_back_uk[temp_back_uk['Win']==1]['drawdown_tick'].mean()
        temp_back_uk['loss']=np.where(temp_back_uk['Pnl']<0, 1, 0)
        loss_uk=temp_back_uk['loss']
        losing_streak_uk = (loss_uk.groupby((loss_uk != loss_uk.shift(1)).cumsum()).cumsum().max())
        mean_uk=temp_back_uk['Pnl'].mean()
        sum_t_uk=temp_back_uk['Pnl'].sum()

        ##################################
        temp_back_us=tradelog_us[tradelog_us['level']==i]

        Win_arr_us = np.array(temp_back_us['Win'])
        number_of_positive_trades_us = np.sum(Win_arr_us)


        #Finding negative trades
        number_of_negative_trades_us=len(temp_back_us['Win'])-number_of_positive_trades_us

        #Finding total trades
        total_trades_us=len(temp_back_us['Win'])


        #Finding max profit in all trades
        max_value_us = temp_back_us['Pnl'].max()
        #print(number_of_positive_trades,number_of_negative_trades,total_trades,max_value)

        #Finding min profit in all trades
        min_value_us = temp_back_us['Pnl'].min()


        temp_back_us['win']=np.where(temp_back_us['Pnl']>=0, 1, 0)
        #df3=df3.dropna()
    #     print(len(df3))
        win_us=temp_back_us['win']
        win_streak_us = (win_us.groupby((win_us != win_us.shift(1)).cumsum()).cumsum().max())
        average_drawdown_us=temp_back_us['drawdown_tick'].mean()
        average_drawdown_win_us=temp_back_us[temp_back_us['Win']==1]['drawdown_tick'].mean()
        temp_back_us['loss']=np.where(temp_back_us['Pnl']<0, 1, 0)
        loss_us=temp_back_us['loss']
        losing_streak_us = (loss_us.groupby((loss_us != loss_us.shift(1)).cumsum()).cumsum().max())
        mean_us=temp_back_us['Pnl'].mean()
        sum_t_us=temp_back_us['Pnl'].sum()

        backtest_lstm[i]["No. of Positive Trades All"]=number_of_positive_trades
        backtest_lstm[i]["No. of Negative Trades All"]=number_of_negative_trades
        backtest_lstm[i]["Win percent All"]=number_of_positive_trades/(number_of_positive_trades+number_of_negative_trades)
        backtest_lstm[i]["Maximum Value All"]=max_value
        backtest_lstm[i]["Minimum Value All"]=min_value
        backtest_lstm[i]["Total Trades All"]=(number_of_positive_trades+number_of_negative_trades)
        backtest_lstm[i]["Win Streak"]=win_streak
        backtest_lstm[i]["Losing Streak"]=losing_streak
        backtest_lstm[i]["Average Drawdown"]=average_drawdown
        backtest_lstm[i]["Average Win Drawdown"]=average_drawdown_win
        backtest_lstm[i]["Total Profit All"]=sum_t
        backtest_lstm[i]["Average Profit All"]=mean

        backtest_lstm[i]["No. of Positive Trades All_asia"]=number_of_positive_trades_asia
        backtest_lstm[i]["No. of Negative Trades All_asia"]=number_of_negative_trades_asia
        backtest_lstm[i]["Win percent All_asia"]=number_of_positive_trades_asia/(number_of_positive_trades_asia+number_of_negative_trades_asia)
        backtest_lstm[i]["Maximum Value All_asia"]=max_value_asia
        backtest_lstm[i]["Minimum Value All_asia"]=min_value_asia
        backtest_lstm[i]["Total Trades All_asia"]=(number_of_positive_trades_asia+number_of_negative_trades_asia)
        backtest_lstm[i]["Win Streak_asia"]=win_streak_asia
        backtest_lstm[i]["Losing Streak_asia"]=losing_streak_asia
        backtest_lstm[i]["Average Drawdown_asia"]=average_drawdown_asia
        backtest_lstm[i]["Average Win Drawdown_asia"]=average_drawdown_win_asia
        backtest_lstm[i]["Total Profit All_asia"]=sum_t_asia
        backtest_lstm[i]["Average Profit All_asia"]=mean_asia

        backtest_lstm[i]["No. of Positive Trades All_uk"]=number_of_positive_trades_uk
        backtest_lstm[i]["No. of Negative Trades All_uk"]=number_of_negative_trades_uk
        backtest_lstm[i]["Win percent All_uk"]=number_of_positive_trades_uk/(number_of_positive_trades_uk+number_of_negative_trades_uk)
        backtest_lstm[i]["Maximum Value All_uk"]=max_value_uk
        backtest_lstm[i]["Minimum Value All_uk"]=min_value_uk
        backtest_lstm[i]["Total Trades All_uk"]=(number_of_positive_trades_uk+number_of_negative_trades_uk)
        backtest_lstm[i]["Win Streak_uk"]=win_streak_uk
        backtest_lstm[i]["Losing Streak_uk"]=losing_streak_uk
        backtest_lstm[i]["Average Drawdown_uk"]=average_drawdown_uk
        backtest_lstm[i]["Average Win Drawdown_uk"]=average_drawdown_win_uk
        backtest_lstm[i]["Total Profit All_uk"]=sum_t_uk
        backtest_lstm[i]["Average Profit All_uk"]=mean_uk

        backtest_lstm[i]["No. of Positive Trades All_us"]=number_of_positive_trades_us
        backtest_lstm[i]["No. of Negative Trades All_us"]=number_of_negative_trades_us
        backtest_lstm[i]["Win percent All_us"]=number_of_positive_trades_us/(number_of_positive_trades_us+number_of_negative_trades_us)
        backtest_lstm[i]["Maximum Value All_us"]=max_value_us
        backtest_lstm[i]["Minimum Value All_us"]=min_value_us
        backtest_lstm[i]["Total Trades All_us"]=(number_of_positive_trades_us+number_of_negative_trades_us)
        backtest_lstm[i]["Win Streak_us"]=win_streak_us
        backtest_lstm[i]["Losing Streak_us"]=losing_streak_us
        backtest_lstm[i]["Average Drawdown_us"]=average_drawdown_us
        backtest_lstm[i]["Average Win Drawdown_us"]=average_drawdown_win_us
        backtest_lstm[i]["Total Profit All_us"]=sum_t_us
        backtest_lstm[i]["Average Profit All_us"]=mean_us
    #tradelog.to_csv('I:\GGNQuants\ScalpingLogic/tradelog_es_30m_scalping_exit_hour_max_level_v2.csv')
    #backtest_lstm.to_csv('I:\GGNQuants\VWAP_STDEV/results/backtest_vwap_'+str(comb[0])+'_sd_'+str(comb[1])+str(m))
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import plotly.express as px
    #df=pd.read_csv("I:\GGNQuants\VWAP_numpy\pandas\cl/tradelog_vwap_5m_rolling_tempCLZ2-Z3.csv")
    df_tradelog=tradelog
    df_tradelog['Entry Time'] =  pd.to_datetime(df_tradelog['Entry Time'])
    df_tradelog['Total_Profit'] = df_tradelog['Pnl'].cumsum()

    df_tradelog['Date'] = pd.to_datetime(df_tradelog['Entry Time']).dt.date
    Day_profit=df_tradelog.groupby(['Date'])["Pnl"].sum()
    Day_min=df_tradelog.groupby(['Date'])["Pnl"].min()
    Day_max=df_tradelog.groupby(['Date'])["Pnl"].max()
    Day_df=pd.concat([Day_profit,Day_min, Day_max], axis=1)

    Day_df.columns=["Pnl","Pnl_min","Pnl_max"]
    Day_df['Total_Profit'] = Day_df['Pnl'].cumsum()
    
    
    hour_profit=df_tradelog.groupby(['hour'])["Pnl"].sum()
    hour_min=df_tradelog.groupby(['hour'])["Pnl"].min()
    hour_max=df_tradelog.groupby(['hour'])["Pnl"].max()
    hour_df=pd.concat([hour_profit,hour_min, hour_max], axis=1)


    hour_df.columns=["Pnl","Pnl_min","Pnl_max"]
    hour_df['Total_Profit'] = hour_df['Pnl'].cumsum()
    
    
    
    

# **************************************** Ploting the Backtested Results *****************************************************************








    fig = px.line(df_tradelog, x="Entry Time", y="Total_Profit")

    fig2 = px.line(Day_df, x=Day_df.index, y="Total_Profit")

    fig3 = go.Figure()
    fig3.add_trace(go.Bar(x=Day_df['Pnl_min'].values,
                            y=Day_df.index,
                            orientation='h',
                            name='Day Minimum',
                            customdata=Day_df['Pnl_min']))
    fig3.add_trace(go.Bar(x= Day_df['Pnl_max'],
                            y =Day_df.index,
                            orientation='h',
                            name='DAY Maximum'))    

    fig3.update_layout(barmode='relative', 
                    height=700, 
                    width=1100, 
                    yaxis_autorange='reversed',
                    bargap=0.01,
                    legend_orientation ='h',
                    legend_x=-0.05, legend_y=1.1
                    )





    fig.write_html(path_for_results+"graph\\"+"new_4_8_new_yearly_equity_vwap_"+str(list_files[m][1])+'max_level'+str(list_files[m][2])+'stoploss_ticks'+str(stoploss_tick)+".html")
    fig2.write_html(path_for_results+"graph\\"+"new_4_8_new_yearly_equity_daily_vwap_"+str(list_files[m][1])+'max_level'+str(list_files[m][2])+'stoploss_ticks'+str(stoploss_tick)+".html")
        
    # Ensure that datetime columns are timezone-unaware before exporting to Excel
    global_tradeLog['Entry Time'] = global_tradeLog['Entry Time'].dt.tz_localize(None)
    global_tradeLog['Exit Time'] = global_tradeLog['Exit Time'].dt.tz_localize(None)
    
    with pd.ExcelWriter(path_for_results+str(m)[:-4]+f'new60v4_year_stoploss_daily_250_fixedtime_{params_list[2]}_{params_list[8]}_{params_list[9]}_level_tick_'+str(list_files[m][1])+'max_level'+str(list_files[m][2])+'stoploss_ticks'+str(stoploss_tick)+'V11.xlsx') as writer:
        global_tradeLog.to_excel(writer, sheet_name="tradelog")
        backtest_lstm.to_excel(writer, sheet_name="backtest")
        
        Day_df.to_excel(writer, sheet_name="daily_min_max")
        hour_df.to_excel(writer, sheet_name="hour_min_max")
        
        
    
    print('yes1')

    end=time.time()
    print(start-end)


In [1]:
# stoploss_tick_inside=5
# time_interval=15
# time_interval_text='15T'
# pnl_stop=-250
# secondary_pnl_stop=-200
# max_stoploss=4
# l=4
params = {
    "stoploss_tick_inside": 5,
    "time_interval": 15,
    "time_interval_text": '15T',
    "pnl_stop": -250,
    "secondary_pnl_stop": -200,
    "max_stoploss": 4,
    "l": 4,
    "stoploss_tick":37
}
params_list=list(params.values())

In [4]:
####### diffrent new approach 
##### updated version 
def trade_logic(day_data,m,tick_stoploss,list_file,params_list):
    stoploss_tick_inside=params_list[0]
    time_interval=params_list[1]
    time_interval_text=params_list[2]
    pnl_stop=params_list[3]
    secondary_pnl_stop=params_list[4]
    max_stoploss=params_list[5]
    l=params_list[6]
    list_files=list_file
    column_names = ['hour',"Type", "Entry Index", 'Entry Time', "Stoploss Price","Entry Price",'level',"Exit Index","Exit Time","Exit Price","Predicted","Trade_close"]
    temp_tradelog = pd.DataFrame(columns = column_names,index=range(5000))
    temp_trade_count=0
    buy1=0
    sell1=0
    stoploss_tick=tick_stoploss
    
    data1=day_data
    data1=data1.reset_index()
    

    print(data1.iloc[0]['Time'])
    stop_buy=0
    stop_sell=0
    algo_stop=0
    
    stoploss_count=0
    for index1,j in data1.iterrows():
        trade_hour=(data1['Time'][index1]).hour
        if algo_stop==1:
            break
        
        if not(datetime.time(0, 0)<=data1['Time'][index1].time()<=datetime.time(0, 5)) and len(df[((df['Date-Time']>=data1['Time'][index1]) & (df['Date-Time']<=data1['Time'][index1]+timedelta(minutes=time_interval)))])!=0  and 14<=trade_hour<=18 and index1<len(data1) :# in temp_list:
            
            new_df=(df[((df['Date-Time']>=data1['Time'][index1]) & (df['Date-Time']<=data1['Time'][index1]+timedelta(minutes=time_interval)))])
            temp_df=new_df
            temp_df=temp_df.dropna()

            temp_df['buy']=0
            temp_df['sell']=0




            tick_size=list_files[m][0]
            max_level=list_files[m][2]
            slippage=1*tick_size
            level_tick=list_files[m][1]
            vwapPrice = data1['pred_close'][index1]
            mid_close=(round(vwapPrice/tick_size))*tick_size
            pred_high=((round((data1['pred_high'][index1])/tick_size))*tick_size)
            pred_low=((round((data1['pred_low'][index1])/tick_size))*tick_size)

            buy_temp=0
            sell_temp=0
            trade_list=[]
            trade_stop=0


#             sd_tick=data1['SD'][index1]
#             sd_tick_limit=3
            for index,i in temp_df.iterrows():
                if temp_df['Date-Time'][index].floor(freq='1T') >= data1['Time'][index1]+timedelta(minutes=1) or (buy1!=0 or sell1!=0):
      
                    price_tick=round(i['Price'],2)
                    bid_price=round(i['Bid Price'],2)
                    ask_price=round(i['Ask Price'],2)
                    mid_close=round(mid_close,2)
                    pred_high=round(pred_high,2)
                    pred_low=round(pred_low,2)

                    trade_buy=int((((price_tick-mid_close)/tick_size)/level_tick))#int(math.floor(((price_tick-mid_close)/tick_size)/level_tick))



        #
                    if (buy1!=0 or sell1!=0) and (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)!=data1['Time'][index1]) and data['Time'][index1].time()==datetime.time(18, 29):
                        temp_df['sell'].loc[index:]=0
                        temp_df['buy'].loc[index:]=0
                        sell1=0
                        buy1=0
                        temp_trade_count-=1
                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                        temp_tradelog['Exit Price'][temp_trade_count]=(mid_close)
                        temp_tradelog['Trade_close'][temp_trade_count]='Day End'


                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                        temp_trade_count+=1
                        trade_stop=1
    #                     if sd_tick>=sd_tick_limit:
    #                         trade_stop=1
                        algo_stop=1



                    if temp_df['sell'][index]>0 and ask_price<=mid_close and sell1!=0 and (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)==data1['Time'][index1]):
                        #if (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = 'T')==data['Time'][index1]):
                        temp_df['sell'].loc[index:]=0
                        temp_df['buy'].loc[index:]=0
                        sell1=0
                        buy1=0
                        temp_trade_count-=1
                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                        temp_tradelog['Exit Price'][temp_trade_count]=(mid_close)
                        temp_tradelog['Trade_close'][temp_trade_count]='Predicted'


                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                        temp_trade_count+=1
                        trade_stop=1
                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                        Pnl_sum=temp_tradelog["Pnl"].sum()
                        if Pnl_sum<=pnl_stop:
                            algo_stop=1
                            break
                        if trade_hour==18:
                            algo_stop=1




                    if temp_df['buy'][index]>0 and bid_price>=mid_close and buy1!=0 and (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)==data1['Time'][index1]):

                        temp_df['buy'].loc[index:]=0
                        temp_df['sell'].loc[index:]=0
                        buy1=0
                        sell=0
                        temp_trade_count-=1
                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                        temp_tradelog['Exit Price'][temp_trade_count]=(mid_close)
                        temp_tradelog['Trade_close'][temp_trade_count]='Predicted'

                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                        temp_trade_count+=1
                        trade_stop=1
    #                     if sd_tick>=sd_tick_limit:
    #                         trade_stop=1
                        if trade_hour==18:
                            algo_stop=1
                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                        Pnl_sum=temp_tradelog["Pnl"].sum()
                        if Pnl_sum<=pnl_stop:
                            algo_stop=1
                            break
                    if (buy1!=0 or sell1!=0) and (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)==data1['Time'][index1]):
                        if sell1!=0:
                                    #print(1,i['Date-Time'],temp_df)
                            #if temp_tradelog['Entry Price'].iloc[temp_trade_count-1]>=mid_close:
                                #if temp_df['Price'].iloc[0]<mid_close:
                                    if ask_price>temp_tradelog['Stoploss Price'].iloc[temp_trade_count-1]+stoploss_tick_inside*tick_size:
                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=ask_price#(temp_tradelog['Stoploss Price'].iloc[temp_trade_count])
                                        temp_tradelog['Trade_close'][temp_trade_count]='stoploss1'


                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        stop_sell=1
                                        trade_stop=1
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                        stoploss_count+=1
                                        if stoploss_count>=max_stoploss and Pnl_sum<=secondary_pnl_stop:
                                            algo_stop=1
                                            break
                               
                                            




                        if buy1!=0:
                            #if temp_tradelog['Entry Price'].iloc[temp_trade_count-1]<=mid_close:
                                #if temp_df['Price'].iloc[0]<=mid_close:
                                    #print(bid_price,index , mid_close,i['Date-Time'])
                                    if bid_price<temp_tradelog['Stoploss Price'].iloc[temp_trade_count-1]-stoploss_tick_inside*tick_size:
                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=bid_price#(temp_tradelog['Stoploss Price'].iloc[temp_trade_count])
                                        temp_tradelog['Trade_close'][temp_trade_count]='stoploss1'


                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        stop_buy=1
                                        trade_stop=1
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                        stoploss_count+=1
                                        if stoploss_count>=max_stoploss and Pnl_sum<=secondary_pnl_stop:
                                            algo_stop=1
                                            break
                    

                    if (buy1!=0 or sell1!=0) and (temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)!=data1['Time'][index1]):
                        if sell1!=0:
                                    #print(1,i['Date-Time'],temp_df)
                            #if temp_tradelog['Entry Price'].iloc[temp_trade_count-1]>=mid_close:
                                #if temp_df['Price'].iloc[0]<mid_close:
                                    if ask_price>=temp_tradelog['Stoploss Price'].iloc[temp_trade_count-1]:
                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=ask_price#(temp_tradelog['Stoploss Price'].iloc[temp_trade_count])
                                        temp_tradelog['Trade_close'][temp_trade_count]='stoploss'


                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        stop_sell=1
                                        trade_stop=1
    #                                     if sd_tick>=sd_tick_limit:
    #                                         trade_stop=1
                                        if trade_hour==18:
                                            algo_stop=1
                                            break
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                        stoploss_count+=1
                                        if stoploss_count>=max_stoploss and Pnl_sum<=secondary_pnl_stop:
                                            algo_stop=1
                                            break

                                    elif ask_price <= temp_tradelog['Predicted'].iloc[temp_trade_count-1]:

                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=(ask_price)
                                        temp_tradelog['Trade_close'][temp_trade_count]='Predicted_other'


                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        trade_stop=1
    #                                     if sd_tick>=sd_tick_limit:
    #                                         trade_stop=1    
                                        if trade_hour==18:
                                            algo_stop=1
                                            break
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                            


                        if buy1!=0:
                            #if temp_tradelog['Entry Price'].iloc[temp_trade_count-1]<=mid_close:
                                #if temp_df['Price'].iloc[0]<=mid_close:
                                    #print(bid_price,index , mid_close,i['Date-Time'])
                                    if bid_price<=temp_tradelog['Stoploss Price'].iloc[temp_trade_count-1]:
                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=bid_price#(temp_tradelog['Stoploss Price'].iloc[temp_trade_count])
                                        temp_tradelog['Trade_close'][temp_trade_count]='stoploss'


                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        stop_buy=1
                                        trade_stop=1
    #                                     if sd_tick>=sd_tick_limit:
    #                                         trade_stop=1
                                        if trade_hour==18:
                                            algo_stop=1
                                            break
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                        stoploss_count+=1
                                        if stoploss_count>=max_stoploss and Pnl_sum<=secondary_pnl_stop:
                                            algo_stop=1
                                            break
                 
                                    elif bid_price >= temp_tradelog['Predicted'].iloc[temp_trade_count-1]:

                                        temp_df['buy'].loc[index:]=0
                                        temp_df['sell'].loc[index:]=0
                                        buy1=0
                                        sell1=0
                                        temp_trade_count-=1
                                        temp_tradelog['Exit Index'][temp_trade_count]=(index)
                                        temp_tradelog['Exit Time'][temp_trade_count]=(temp_df['Date-Time'][index])
                                        temp_tradelog['Exit Price'][temp_trade_count]=(bid_price)
                                        temp_tradelog['Trade_close'][temp_trade_count]='Predicted_other'

                                        temp_tradelog=temp_tradelog.bfill(axis ='rows')

                                        temp_trade_count+=1
                                        trade_stop=1
    #                                     if sd_tick>=sd_tick_limit:
    #                                         trade_stop=1
                                        if trade_hour==18:
                                            algo_stop=1
                                            break
                                        temp_tradelog.loc[temp_tradelog['Type']=='buy',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='buy']['Exit Price']-temp_tradelog[temp_tradelog['Type']=='buy']['Entry Price'])/tick_size)
                                        temp_tradelog.loc[temp_tradelog['Type']=='sell',"Pnl"]=((temp_tradelog[temp_tradelog['Type']=='sell']['Entry Price']-temp_tradelog[temp_tradelog['Type']=='sell']['Exit Price'])/tick_size)
                                        Pnl_sum=temp_tradelog["Pnl"].sum()
                                        if Pnl_sum<=pnl_stop:
                                            algo_stop=1
                                            break
                                        stoploss_count+=1
                                        if stoploss_count>=max_stoploss:
                                            algo_stop=1
                                            break


                    if bid_price-mid_close>=0 and trade_hour!=18 and stop_sell==0 and df['Date-Time'][index].floor(freq = time_interval_text).time()>=datetime.time(14, 30) and trade_stop==0 and bid_price<=pred_high and not(datetime.time(23, 50)<=data1['Time'][index1].time()<=datetime.time(23, 59)):
                        trade_buy=int((round((bid_price-mid_close)/tick_size)/level_tick))#int(math.floor(((price_tick-mid_close)/tick_size)/level_tick))

                        if trade_buy>=1 and trade_buy>temp_df['sell'][index] and trade_buy<=max_level and (temp_trade_count==0 or temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)==data1['Time'][index1] or math.isnan(temp_tradelog['Exit Price'].iloc[temp_trade_count-1])==False): 
                            if temp_df['sell'].loc[index]==0 and trade_buy>3:
                                for x in range(l,trade_buy+1):
                                    #print(trade_buy,x)
                                    temp_df['sell'].loc[index:]=x
                                    temp_tradelog['hour'][temp_trade_count]=(trade_hour)
                                    temp_tradelog['Type'][temp_trade_count]=('sell')
                                    temp_tradelog['Entry Index'][temp_trade_count]=(index)
                                    temp_tradelog['Entry Time'][temp_trade_count]=(df['Date-Time'][index])
                                    temp_tradelog['Entry Price'][temp_trade_count] = mid_close + (x * tick_size * level_tick)
                                    temp_tradelog['Stoploss Price'][temp_trade_count] = mid_close+stoploss_tick*tick_size
                                    temp_tradelog['level'][temp_trade_count]=x
                                    temp_tradelog['Predicted'][temp_trade_count]=mid_close


                                    temp_trade_count+=1

                                    sell1=1
                                    stop_buy=0
                            elif  trade_buy>=l:

                                temp_df['sell'].loc[index:]=trade_buy
                                temp_tradelog['hour'][temp_trade_count]=(trade_hour)
                                temp_tradelog['Type'][temp_trade_count]=('sell')
                                temp_tradelog['Entry Index'][temp_trade_count]=(index)
                                temp_tradelog['Entry Time'][temp_trade_count]=(df['Date-Time'][index])
                                temp_tradelog['Entry Price'][temp_trade_count] = mid_close + (trade_buy * tick_size * level_tick)
                                temp_tradelog['Stoploss Price'][temp_trade_count] = mid_close  +stoploss_tick*tick_size
                                temp_tradelog['level'][temp_trade_count]=trade_buy
                                temp_tradelog['Predicted'][temp_trade_count]=mid_close

                                temp_trade_count+=1

                                sell1=1
                                stop_buy=0




                    if ask_price-mid_close<=0 and trade_hour!=18 and stop_buy==0 and trade_stop==0 and df['Date-Time'][index].floor(freq = time_interval_text).time()>=datetime.time(14, 30) and ask_price>=pred_low and not(datetime.time(23, 50)<=data1['Time'][index1].time()<=datetime.time(23, 59)):

                        trade_buy=int((round((ask_price-mid_close)/tick_size)/level_tick))#int(math.floor(((price_tick-mid_close)/tick_size)/level_tick))

                        trade_buy=-trade_buy

                        if trade_buy>=1 and trade_buy>temp_df['buy'][index] and trade_buy<=max_level and (temp_trade_count==0 or temp_tradelog['Entry Time'].iloc[temp_trade_count-1].floor(freq = time_interval_text)==data1['Time'][index1] or math.isnan(temp_tradelog['Exit Price'].iloc[temp_trade_count-1])==False ):
                            if temp_df['buy'].loc[index]==0 and trade_buy>3:

                                for x in range(l,trade_buy+1):

                                    temp_df['buy'].loc[index:]=x
                                    temp_tradelog['hour'][temp_trade_count]=(trade_hour)
                                    temp_tradelog['Type'][temp_trade_count]=('buy')
                                    temp_tradelog['Entry Index'][temp_trade_count]=(index)
                                    temp_tradelog['Entry Time'][temp_trade_count]=(df['Date-Time'][index])
                                    temp_tradelog['Entry Price'][temp_trade_count] = mid_close - (x * tick_size * level_tick)
                                    temp_tradelog['Stoploss Price'][temp_trade_count] = mid_close -stoploss_tick*tick_size  
                                    temp_tradelog['level'][temp_trade_count]=x
                                    temp_tradelog['Predicted'][temp_trade_count]=mid_close

                                    temp_trade_count+=1
                                    buy1=1
                                    stop_sell=0
                            elif trade_buy>=l:
                                temp_df['buy'].loc[index:]=trade_buy
                                temp_tradelog['hour'][temp_trade_count]=(trade_hour)
                                temp_tradelog['Type'][temp_trade_count]=('buy')
                                temp_tradelog['Entry Index'][temp_trade_count]=(index)
                                temp_tradelog['Entry Time'][temp_trade_count]=(df['Date-Time'][index])
                                temp_tradelog['Entry Price'][temp_trade_count] = mid_close - (trade_buy * tick_size * level_tick)
                                temp_tradelog['Stoploss Price'][temp_trade_count] = mid_close -stoploss_tick*tick_size  
                                temp_tradelog['level'][temp_trade_count]=trade_buy
                                temp_tradelog['Predicted'][temp_trade_count]=mid_close

                                temp_trade_count+=1
                                buy1=1
                                stop_sell=0



    temp_tradelog=temp_tradelog.dropna()
    return temp_tradelog
        